In [1]:
import tensorflow as tf
import os
from tensorflow.keras.applications import MobileNet  # Import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir = "E:/Dataset/IK_pcon"
train_dir = os.path.join(data_dir, "train")

img_size = 640  # MobileNet default image size is 224x224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.8, 1.2],
    shear_range=0.2,
    channel_shift_range=20,
    validation_split=0.2,
)

train_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Create the base model from the pre-trained MobileNet
base_model = MobileNet(include_top=False, weights='imagenet', input_shape=(img_size, img_size, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)  # Using Dropout to prevent overfitting
predictions = Dense(1, activation='sigmoid')(x)

# Compile the model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model with the same settings
history = model.fit(
    train_set,
    epochs=50,
    validation_data=val_set
)


Found 5888 images belonging to 2 classes.
Found 1472 images belonging to 2 classes.


C:\Users\parkpaul\AppData\Local\Temp\ipykernel_22408\2508329228.py:39: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNet(include_top=False, weights='imagenet', input_shape=(img_size, img_size, 3))


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/50


C:\Users\parkpaul\AppData\Local\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


184/184 ━━━━━━━━━━━━━━━━━━━━ 2867s 15s/step - accuracy: 0.5606 - loss: 0.7071 - val_accuracy: 0.5611 - val_loss: 0.6753
Epoch 2/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 2825s 15s/step - accuracy: 0.6592 - loss: 0.6219 - val_accuracy: 0.5971 - val_loss: 0.6576
Epoch 3/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 2766s 15s/step - accuracy: 0.6738 - loss: 0.6021 - val_accuracy: 0.5944 - val_loss: 0.6524
Epoch 4/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 2761s 15s/step - accuracy: 0.6910 - loss: 0.5810 - val_accuracy: 0.6393 - val_loss: 0.6247
Epoch 5/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 2663s 14s/step - accuracy: 0.6948 - loss: 0.5686 - val_accuracy: 0.6304 - val_loss: 0.6206
Epoch 6/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 2786s 15s/step - accuracy: 0.6954 - loss: 0.5676 - val_accuracy: 0.5904 - val_loss: 0.6738
Epoch 7/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 2817s 15s/step - accuracy: 0.7065 - loss: 0.5626 - val_accuracy: 0.6073 - val_loss: 0.6544
Epoch 8/50
184/184 ━━━━━━━━━━━━━━━━━━━━ 2654s 14s/step - accuracy: 0.7035 - loss: 0.5622 - val

184/184 ━━━━━━━━━━━━━━━━━━━━ 2681s 14s/step - accuracy: 0.7971 - loss: 0.4398 - val_accuracy: 0.7418 - val_loss: 0.5114


In [2]:
model.save('IK_pcon5.keras')